# Spotify Audio Features

In [ ]:
%load_ext autoreload
%autoreload 2

## Fetch user track features

### Spotify API Client & Authentication

In [ ]:
import spotipy

In [ ]:
ACCESS_TOKEN = "BQDwcxQZngYDU1IEgtnB_v_8jzYYjPJ4QXFUzChone47-qjGi6Yp7ke2_sVk2t3wNoUCub2b4VM0xbagq3U6TCDmdbjgYNtOC1XZWRU7_udemTLlY4QF2gabcBkWsHK3l2j0WA_rDZ_ao6VKuzXNPDNYEu2knZNC2P31sB-doeI"
spotify = spotipy.Spotify(auth=ACCESS_TOKEN, requests_timeout=15)

### Fetch current user's tracks

In [ ]:
track_list = []
has_next = True
while has_next:
    result = spotify.current_user_saved_tracks(limit=50, offset=len(track_list))
    track_list.extend([user_track["track"] for user_track in result["items"]])
    has_next = bool(result["next"])
print(len(track_list))

### Query Spotify API for audio features

In [ ]:
from itertools import chain
from more_itertools import chunked

# Extract track IDs so we can fetch additional info
track_id_list = [track["id"] for track in track_list]
# Get track audio features
track_features_list = list(chain.from_iterable(spotify.audio_features(track_ids) for track_ids in chunked(track_id_list, 50)))
# Get track audio analysis
# track_analysis_list = [spotify.audio_analysis(track_id) for track_id in track_id_list]
import asyncio
loop = asyncio.get_running_loop()
track_analysis_list = await asyncio.gather(*[loop.run_in_executor(None, spotify.audio_analysis, track_id) for track_id in track_id_list])

### Construct & normalize audio features

In [ ]:
from smoothify.features import construct_features

In [ ]:
features_df = construct_features(
    audio_features_list=track_features_list,
    audio_analysis_list=track_analysis_list
)
features_df

## Compute the 'best' order for the user's tracks

In [ ]:
import numpy as np
import scipy.spatial

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

### Visualize the distribution of pairwise distances

In [ ]:
NORM_P = 2
points = features_df.to_numpy()
print(points.shape)
distance_matrix = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(points, "minkowski", p=NORM_P))
sns.displot(distance_matrix.flatten())

### Define the Annealer

In [ ]:
from simanneal import Annealer

rng = np.random.default_rng()

class BottleneckAnnealer(Annealer):
    copy_strategy = "method"  # Use `self.state.copy()` to copy the state
    
    # Override default hyperparameters
    Tmin = 1e-3
    Tmax = 5e3
    steps = 1000000
    updates = 1000

    def __init__(self, *args, path: np.ndarray, distance_matrix: np.ndarray, **kwargs):
        super().__init__(path, *args, **kwargs)
        self.distance_matrix = distance_matrix
        self.num_points = len(self.state)

    def move(self):
        """
        Randomly swap points
        """
        num_swaps = int(np.random.rand() * 4)
        swaps = rng.choice(self.num_points, (num_swaps, 2), replace=False)
        self.state[swaps] = self.state[swaps[..., ::-1]]

    def energy(self):
        """
        Compute the energy of the current path
        """
        # Find the length of the edge from each node in the path to the next
        source_nodes = self.state
        target_nodes = np.roll(source_nodes, 1)
        edge_distances = self.distance_matrix[source_nodes, target_nodes]
        max_edge_length = edge_distances.max()
#         mean_edge_length = edge_distances.mean()
        energy = max_edge_length
        return energy

### Optimize

In [ ]:
annealer = BottleneckAnnealer(path=np.arange(len(points)), distance_matrix=distance_matrix)
print(f"Start energy: {annealer.energy()}")

best_path, best_energy = annealer.anneal()
print(f"Best energy: {best_energy}")
print(best_path)

In [ ]:
features_df.iloc[best_path]

In [ ]:
optimally_ordered_tracks = [track_list[idx]["uri"] for idx in best_path]
for url in optimally_ordered_tracks:
    print(url)